__File: src_2021_workshop_fig.ipynb__

__Author:   A. J. Tropiano (tropiano.4@osu.edu)__<br/>
__Date:     Mar 17, 2021__

Generating some figures for Scott's talk in the JLab SRC EMC workshop. Also use this for testing SRC functions and related things.

__Revision history:<br/>__
* ...

In [ ]:
# import more things as you create functions

import numpy as np
# Scripts made by A.T.
from Figures import figures_functions as ff
from Potentials.vsrg_macos import vnn
from SRG.srg_unitary_transformation import SRG_unitary_transformation

In [ ]:
# Strategy: let's import functions and classes from the various testing scripts. Once things are ironed out, you
# can create official scripts with functions and classes, moving the test_scripts to Old_codes.

In [ ]:
# a_2 scaling factor calculations

# \int_2^{\infty} dq P^A(q) / \int_2^{\infty} dq P^d(q) vs A
# print values as well

In [ ]:
# decomposition of n_{\lambda}^d(q) into 1, \deltaU, \deltaU \deltaU^{\dagger}

In [ ]:
# 3S1 / 1S0 ratios of \deltaU \deltaU^{\dagger} term as a function of q

In [ ]:
# pp / pn ratios of < n_{\lambda}^A(q, Q=0) > as a function of q

# gray out q < \lambda region or don't plot it
# alternatively, only show high-q contribution?

In [ ]:
# bar chart showing contributions from 3S1-3S1, 3S1-3D1, 3D1-3D1 to deuteron in bare vs evolved

# copy figure from src notebook

In [ ]:
# ratio of < n_{\lambda}^d(q) > ( 3S1 only / (3S1+3D1) )

In [ ]:
# factorization of \deltaU \deltaU^{\dagger}

# do \deltaU(k_i, q) \deltaU^{\dagger}(q, k_i) / \deltaU(k_0, q) \deltaU^{\dagger}(q, k_0) similar to other ways
# you've done this